# Нечеткие автоматы

In [1]:
import numpy as np
import torch

In [2]:
import sys
sys.path.insert(0, './python')

In [3]:
from fuzzy_torch import logic
from fuzzy_torch.modules import ffsa, indicators

## Проверка переходов

In [4]:
Logic = logic.Hamacher
fuzzy_fsa = ffsa.TimeDependentFFSA(Logic)

In [5]:
# Добавление состояний и переходов.
fuzzy_fsa.states = [0, 1]
fuzzy_fsa.transitions.append(ffsa.FuzzyTransitionContinuous(0, 1, indicators.Sigmoid(1, 1.0, 0.0), 2))

In [6]:
# Начальные активации.
activation = torch.tensor(np.array([[1.0, 0.0],
                                    [1.0, 0.0],
                                    [1.0, 0.0],
                                    [1.0, 0.0]]).astype(np.float32), requires_grad=True)

# Последовательности.
sequence = torch.tensor(np.array([[-1.0, -1.0, 0.0, 1.0, 1.0],
                                  [-10.0, -5.0, -1.0, 0.0, 10.0],
                                  [-10.0, -10.0, -10.0, -10.0, 0.0],
                                  [-0.5, -0.5, -0.5, -0.5, -0.5]]).astype(np.float32), requires_grad=True)

In [7]:
Activations = [activation]
for step in range(sequence.size()[1]):
    Activations.append(fuzzy_fsa(sequence[:, step][:,None], Activations[-1], 0.05))

In [8]:
torch.sum(Activations[-1][:,1]).backward()

In [9]:
Activations

[tensor([[1., 0.],
         [1., 0.],
         [1., 0.],
         [1., 0.]], requires_grad=True), tensor([[9.7311e-01, 2.6894e-02],
         [1.0000e+00, 4.5398e-06],
         [1.0000e+00, 4.5398e-06],
         [9.6225e-01, 3.7754e-02]], grad_fn=<ClampBackward1>), tensor([[9.4641e-01, 5.3590e-02],
         [9.9933e-01, 6.7382e-04],
         [9.9999e-01, 9.0796e-06],
         [9.2504e-01, 7.4957e-02]], grad_fn=<ClampBackward1>), tensor([[8.9779e-01, 1.0221e-01],
         [9.7244e-01, 2.7563e-02],
         [9.9999e-01, 1.3619e-05],
         [8.8841e-01, 1.1159e-01]], grad_fn=<ClampBackward1>), tensor([[8.3030e-01, 1.6970e-01],
         [9.2314e-01, 7.6864e-02],
         [9.9998e-01, 1.8159e-05],
         [8.5236e-01, 1.4764e-01]], grad_fn=<ClampBackward1>), tensor([[0.7667, 0.2333],
         [0.8308, 0.1692],
         [0.9500, 0.0500],
         [0.8169, 0.1831]], grad_fn=<ClampBackward1>)]

In [10]:
sequence.grad

tensor([[1.6873e-02, 1.6751e-02, 2.0997e-02, 1.5773e-02, 1.4882e-02],
        [3.9518e-06, 5.7873e-04, 1.7234e-02, 2.1876e-02, 3.8703e-06],
        [4.4261e-06, 4.4261e-06, 4.4261e-06, 4.4261e-06, 2.5000e-02],
        [2.2024e-02, 2.1710e-02, 2.1386e-02, 2.1050e-02, 2.0704e-02]])

In [11]:
print(fuzzy_fsa.transitions[0].condition.linear.weight.grad)
print(fuzzy_fsa.transitions[0].condition.linear.bias.grad)

tensor([[-0.0767]])
tensor([[0.2569]])


## Последовательность с переключением

In [12]:
class SwitchingRegressor(torch.nn.Module):
    def __init__(self, logic, ffsa):
        super().__init__()
        self.logic = logic
        self.ffsa = ffsa
        self.debug = False
        
    def forward(self, input, init_activation):
        steps = input.size()[1]
        
        activations = [init_activation]
        outputs = []
        for step in range(steps):
            # Срез входа по текущему шагу.
            input_on_current_step = input[:, step]
            
            # Новые активации (согласно нечеткому конечному автомату).
            activations.append(self.ffsa(input_on_current_step, activations[-1], 0.05))
            
            # Получение выходов регрессоров.
            output = [state(input_on_current_step) for state in self.ffsa.states]
            output = torch.stack(output, dim=1)
            outputs.append(torch.einsum("bo,bo...->b...", activations[-1], output))
            
        return torch.stack(outputs, dim=1), torch.stack(activations, dim=1)

In [13]:
# Логика.
#Logic = logic.Godel
#Logic = logic.Product
#Logic = logic.Lukasiewicz
#Logic = logic.Nilpotent
Logic = logic.Hamacher

In [14]:
regressor = SwitchingRegressor(Logic, ffsa.TimeDependentFFSA(Logic, normalize=True))

In [15]:
regressor.ffsa.states = torch.nn.ModuleList([torch.nn.Linear(2, 1), torch.nn.Linear(2, 1), torch.nn.Linear(2, 1)])
regressor.ffsa.transitions = torch.nn.ModuleList([ffsa.FuzzyTransitionContinuous(0, 1, indicators.Sigmoid(2)),
                                                  ffsa.FuzzyTransitionContinuous(0, 2, indicators.Sigmoid(2))])

### Набор данных

In [36]:
import scipy.stats as stats

class SwitchingSequences(torch.utils.data.Dataset):
    def __init__(self, length=100, delta=3):       
        self.length = length
        self.delta = delta
        self.noize = stats.norm()
    
    def __len__(self):
        return 16384
    
    def __getitem__(self, idx):
        switch_index_start = np.random.choice(np.arange(0, self.length - self.delta - 1), 1)[0]
        switch_index_end = np.random.choice(np.arange(switch_index_start + self.delta, self.length), 1)[0]
        switch_type = np.random.choice([-1.0, 1.0], 1)[0]
        
        X = np.ones((self.length, 2))
        X[:,1] *= switch_type
        X[:switch_index_start, 1] = 0.0
        X[switch_index_end:, 1] = 0.0
        X += self.noize.rvs(X.shape) * 0.1
        
        y = np.ones((self.length))
        y[switch_index_start:] = 4.0 * switch_type * X[switch_index_start:, 0]
        y += self.noize.rvs(y.shape) * 0.1
        
        return X.astype(np.float32), y.astype(np.float32)[:, None]

In [37]:
dataset = SwitchingSequences()

In [38]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=512,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

In [39]:
import torch.optim as optim

optimizer = optim.Adam(regressor.parameters(), lr=0.1)

In [40]:
loss = torch.nn.MSELoss()

for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        x, true_y = data
        init_activations = torch.zeros(x.size()[0], 3)
        init_activations[:,0] = 1.0

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        y, activations = regressor(x, init_activations)
        eval_loss = loss(y, true_y)
        eval_loss.backward()
        optimizer.step()

        # print statistics
        running_loss += eval_loss.item()
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss:.3f}')
        running_loss = 0.0

print('Finished Training')

[1,     1] loss: 2.377
[1,     2] loss: 1.814
[1,     3] loss: 1.427
[1,     4] loss: 1.365
[1,     5] loss: 1.093
[1,     6] loss: 0.903
[1,     7] loss: 0.780
[1,     8] loss: 0.623
[1,     9] loss: 0.564
[1,    10] loss: 0.557
[1,    11] loss: 0.602
[1,    12] loss: 0.569
[1,    13] loss: 0.517
[1,    14] loss: 0.582
[1,    15] loss: 0.598
[1,    16] loss: 0.593
[1,    17] loss: 0.605
[1,    18] loss: 0.655
[1,    19] loss: 0.517
[1,    20] loss: 0.549
[1,    21] loss: 0.545
[1,    22] loss: 0.528
[1,    23] loss: 0.464
[1,    24] loss: 0.459
[1,    25] loss: 0.357
[1,    26] loss: 0.394
[1,    27] loss: 0.388
[1,    28] loss: 0.363
[1,    29] loss: 0.391
[1,    30] loss: 0.381
[1,    31] loss: 0.347
[1,    32] loss: 0.397
[2,     1] loss: 0.384
[2,     2] loss: 0.364
[2,     3] loss: 0.355
[2,     4] loss: 0.369
[2,     5] loss: 0.348
[2,     6] loss: 0.350
[2,     7] loss: 0.351
[2,     8] loss: 0.312
[2,     9] loss: 0.313
[2,    10] loss: 0.323
[2,    11] loss: 0.285
[2,    12] 

[12,     3] loss: 0.054
[12,     4] loss: 0.060
[12,     5] loss: 0.056
[12,     6] loss: 0.050
[12,     7] loss: 0.057
[12,     8] loss: 0.054
[12,     9] loss: 0.049
[12,    10] loss: 0.051
[12,    11] loss: 0.057
[12,    12] loss: 0.053
[12,    13] loss: 0.050
[12,    14] loss: 0.055
[12,    15] loss: 0.058
[12,    16] loss: 0.053
[12,    17] loss: 0.049
[12,    18] loss: 0.055
[12,    19] loss: 0.056
[12,    20] loss: 0.053
[12,    21] loss: 0.049
[12,    22] loss: 0.053
[12,    23] loss: 0.066
[12,    24] loss: 0.059
[12,    25] loss: 0.058
[12,    26] loss: 0.057
[12,    27] loss: 0.055
[12,    28] loss: 0.054
[12,    29] loss: 0.055
[12,    30] loss: 0.059
[12,    31] loss: 0.051
[12,    32] loss: 0.054
[13,     1] loss: 0.052
[13,     2] loss: 0.054
[13,     3] loss: 0.049
[13,     4] loss: 0.050
[13,     5] loss: 0.051
[13,     6] loss: 0.048
[13,     7] loss: 0.055
[13,     8] loss: 0.051
[13,     9] loss: 0.048
[13,    10] loss: 0.048
[13,    11] loss: 0.052
[13,    12] loss

In [45]:
X, y_true = dataset[0]
X = torch.tensor(X)[None,:]
y, activations = regressor(X, torch.tensor(np.array([[1.0, 0.0, 0.0]]).astype(np.float32)))

print(y_true.squeeze())
print(y.squeeze().detach().numpy())

[ 1.0636816   1.1271845   0.96690947  0.9399483   0.89591223  1.0893327
  0.9902807   1.1074219   0.8486005   0.99929607  0.9036388   0.9770301
  0.9922872   1.0501469   0.8978201   0.88019955  1.0069315   0.8284542
  1.2606331   1.0277216   1.0025424   0.9353081   1.0308428   0.9126508
  0.8804698   0.9993591   0.9339324   1.1085291   1.0614871   1.0736322
  0.8583249   1.2489167   0.7749296   1.0065045   1.21576    -4.212572
 -4.2949495  -4.099417   -4.081355   -4.1245756  -3.1380196  -3.8981912
 -4.034328   -3.8807492  -4.2461705  -3.7869663  -4.31014    -4.0272746
 -4.23155    -3.8829606  -2.9306242  -3.720491   -4.8416324  -4.0280437
 -4.571766   -3.6654613  -4.6328936  -4.0595636  -5.0068264  -3.7584162
 -3.5817647  -4.0784016  -3.202081   -3.4220936  -3.0548804  -4.4677525
 -3.8817286  -3.9379535  -4.1695156  -3.9721007  -4.1572433  -4.9272237
 -4.573931   -3.8898299  -3.8300617  -3.5908124  -3.6964996  -4.5851994
 -4.240703   -3.6830387  -4.0223193  -4.134002   -4.0102105  -3.5

In [46]:
y_true.squeeze() - y.squeeze().detach().numpy()

array([ 0.1417675 ,  0.04074812, -0.07147318, -0.1256578 ,  0.20727146,
        0.17486185,  0.1217016 ,  0.01777899, -0.19497621,  0.01867318,
       -0.20767313,  0.19387001,  0.18125683,  0.14310575, -0.22554839,
       -0.07924098, -0.00226212, -0.16806537,  0.21155012, -0.05691338,
       -0.0588274 , -0.24938774,  0.09071606, -0.2734815 , -0.09011495,
       -0.23064   , -0.1934346 , -0.1532929 ,  0.05424774,  0.07387078,
       -0.37648666,  0.07331002, -0.2154764 ,  0.0497452 ,  0.19882786,
       -0.04247952, -0.03680944,  0.05335665,  0.18088102, -0.002038  ,
        0.06857562, -0.07825065, -0.04867911,  0.0082047 ,  0.08277941,
        0.08407307, -0.02010918,  0.05632019, -0.04808998,  0.01496196,
       -0.00656271,  0.01747179, -0.07677746, -0.00488997, -0.09306145,
       -0.0282681 ,  0.02340126, -0.00829077, -0.10740042, -0.01680779,
       -0.05985308,  0.06244516,  0.01016831,  0.09403157,  0.25331426,
       -0.02494144,  0.11300826,  0.1320281 ,  0.04700327,  0.17

In [47]:
activations.argmax(dim=2)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2]])

In [48]:
activations.sum(dim=2)

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000]], grad_fn=<SumBackward1>)

In [50]:
print("Первое условие (параметры):")
print(regressor.ffsa.transitions[0].speed.detach().numpy())
print(regressor.ffsa.transitions[0].condition.linear.weight.detach().numpy())
print(regressor.ffsa.transitions[0].condition.linear.bias.detach().numpy())

print("Второе условие (параметры):")
print(regressor.ffsa.transitions[1].speed.detach().numpy())
print(regressor.ffsa.transitions[1].condition.linear.weight.detach().numpy())
print(regressor.ffsa.transitions[1].condition.linear.bias.detach().numpy())

Первое условие (параметры):
[6.8892913]
[[-4.028718 11.741827]]
[-3.8700418]
Второе условие (параметры):
[21.689459]
[[ -6.4538674 -21.918095 ]]
[-8.063359]
